In [16]:
import pandas as pd

# Load the dataset
data = pd.read_csv('Test Task - Marketing Measurement - DF.csv', index_col='date')
data.index = pd.to_datetime(data.index)  # Ensure the index is treated as datetime

# Assuming the campaign started on 2023-09-18, split the data into pre and post
pre_campaign = data[:'2023-09-17']
post_campaign = data['2023-09-18':]

# Choose cities for treatment and control groups
treatment_cities = ['Warsaw', 'Krakow', 'Poznan', 'Wroclaw']
control_cities = ['Gdansk', 'Lublin', 'Szczecin', 'Rzeszow']

# Calculate the average number of rides in pre and post campaign periods
avg_pre_treat = pre_campaign[treatment_cities].mean()
avg_post_treat = post_campaign[treatment_cities].mean()
avg_pre_control = pre_campaign[control_cities].mean()
avg_post_control = post_campaign[control_cities].mean()

# DiD calculation for each treatment city
did_estimate = {
    city: (avg_post_treat[city] - avg_pre_treat[city]) - (avg_post_control[control_cities[0]] - avg_pre_control[control_cities[0]])
    for city in treatment_cities
}

# Display the results
print("Pre-Campaign Average (Treatment):", avg_pre_treat)
print("Post-Campaign Average (Treatment):", avg_post_treat)
print("Pre-Campaign Average (Control):", avg_pre_control)
print("Post-Campaign Average (Control):", avg_post_control)

# Display DiD estimates
print("Difference-in-Differences Estimate:", did_estimate)





Pre-Campaign Average (Treatment): Warsaw     1132.3424
Krakow      348.2496
Poznan     1475.5024
Wroclaw     183.4608
dtype: float64
Post-Campaign Average (Treatment): Warsaw      647.0
Krakow      240.0
Poznan     1467.0
Wroclaw     143.0
dtype: float64
Pre-Campaign Average (Control): Gdansk      969.0272
Lublin       88.0240
Szczecin    154.7280
Rzeszow      55.1216
dtype: float64
Post-Campaign Average (Control): Gdansk      779.0
Lublin       60.0
Szczecin    118.0
Rzeszow      56.0
dtype: float64
Difference-in-Differences Estimate: {'Warsaw': -295.3152, 'Krakow': 81.7776, 'Poznan': 181.5247999999999, 'Wroclaw': 149.5664}


In [18]:
import pandas as pd

# Constants for the campaign
CPA_low = 6  # Lower bound of cost per acquisition
CPA_high = 12  # Higher bound of cost per acquisition
CPM_low = 2  # Lower bound of cost per thousand impressions
CPM_high = 4  # Higher bound of cost per thousand impressions
average_profit_per_ride = 6  # Average profit per ride in euros
total_budget = 5000  # Total budget for the campaign in euros


# Calculate total additional rides over the campaign period (assumed 30 days)
campaign_days = 30
total_additional_rides = {city: rides * campaign_days for city, rides in did_estimate.items()}

# Function to calculate costs and revenues
def calculate_costs_and_revenues(additional_rides, CPA, profit_per_ride):
    total_cost = additional_rides * CPA
    total_revenue = additional_rides * profit_per_ride
    return total_cost, total_revenue

# Calculations for each city at low and high CPA
costs_revenues = {}
efficiencies = {}
for city, additional_rides in total_additional_rides.items():
    costs_revenues[city] = {
        'Low CPA': calculate_costs_and_revenues(additional_rides, CPA_low, average_profit_per_ride),
        'High CPA': calculate_costs_and_revenues(additional_rides, CPA_high, average_profit_per_ride)
    }
    # Calculate efficiency
    if costs_revenues[city]['Low CPA'][0] != 0:
        efficiencies[city] = {
            'Low CPA': costs_revenues[city]['Low CPA'][1] / costs_revenues[city]['Low CPA'][0],
            'High CPA': costs_revenues[city]['High CPA'][1] / costs_revenues[city]['High CPA'][0] if costs_revenues[city]['High CPA'][0] != 0 else 'Undefined'
        }

# CPM analysis for estimated impressions
impressions_low = (total_budget / CPM_high) * 1000
impressions_high = (total_budget / CPM_low) * 1000

# Output results
print("Total Additional Rides per City:")
for city, rides in total_additional_rides.items():
    print(f"{city}: {rides}")
print("\nCost and Revenue per City:")
for city, values in costs_revenues.items():
    print(f"{city} - Low CPA: Cost = {values['Low CPA'][0]}, Revenue = {values['Low CPA'][1]}")
    print(f"{city} - High CPA: Cost = {values['High CPA'][0]}, Revenue = {values['High CPA'][1]}")
print("\nEfficiency per City:")
for city, eff in efficiencies.items():
    print(f"{city} - Low CPA: {eff['Low CPA']}")
    print(f"{city} - High CPA: {eff['High CPA']}")

print(f"\nEstimated Impressions (Low CPM estimate): {int(impressions_high)}")
print(f"Estimated Impressions (High CPM estimate): {int(impressions_low)}")


Total Additional Rides per City:
Warsaw: -8859.456
Krakow: 2453.3280000000004
Poznan: 5445.743999999997
Wroclaw: 4486.991999999999

Cost and Revenue per City:
Warsaw - Low CPA: Cost = -53156.736000000004, Revenue = -53156.736000000004
Warsaw - High CPA: Cost = -106313.47200000001, Revenue = -53156.736000000004
Krakow - Low CPA: Cost = 14719.968000000003, Revenue = 14719.968000000003
Krakow - High CPA: Cost = 29439.936000000005, Revenue = 14719.968000000003
Poznan - Low CPA: Cost = 32674.46399999998, Revenue = 32674.46399999998
Poznan - High CPA: Cost = 65348.92799999996, Revenue = 32674.46399999998
Wroclaw - Low CPA: Cost = 26921.951999999997, Revenue = 26921.951999999997
Wroclaw - High CPA: Cost = 53843.903999999995, Revenue = 26921.951999999997

Efficiency per City:
Warsaw - Low CPA: 1.0
Warsaw - High CPA: 0.5
Krakow - Low CPA: 1.0
Krakow - High CPA: 0.5
Poznan - Low CPA: 1.0
Poznan - High CPA: 0.5
Wroclaw - Low CPA: 1.0
Wroclaw - High CPA: 0.5

Estimated Impressions (Low CPM estimat